# Exploring World Internet Access Using SQL

## Introduction

I use PostgreSQL to answer five questions about internet access around the world. The dataset used contains information on internet access, telephone landline connections, broadband internet landline connections, and mobile subscriptions for each country every year for the past few decades. I use SQL techniques like aggregations, joining, window functions, common table expressions, and subqueries to answer five questions using this dataset, providing a data-informed picture of global internet access.

## Data

This dataset contains information on internet access around the world. The data are contained in the following two CSV files:

- `internet_users.csv`
    - `users` - The number of people who have used the internet in the last three months
    - `share` - The share of the entity's population who have used the internet in the last three months
- `adoption.csv`
    - `fixed_telephone_subs` - The number of people who have a telephone landline connection
    - `fixed_telephone_subs_share` - The share of the entity's population who have a telephone landline connection
    - `fixed_broadband_subs` - The number of people who have a broadband internet landline connection
    - `fixed_broadband_subs_share` - The share of the entity's population who have a broadband internet landline connection
    - `mobile_cell_subs` - The number of people who have a mobile subscription
    - `mobile_cell_subs_share` - The share of the entity's population who have a mobile subscription

Both data files also contain the following 3 attributes:

- `entity` - The name of the country, region, or group.
- `code` - Unique id for the country (null for other entities).
- `year` - The year ranging from 1990 - 2020 (in `internet_users.csv`) and from 1960 - 2021 (in `adoption.csv`).


This data comes from [DataCamp Workspace](https://www.datacamp.com/workspace), originally sourced from [Our World In Data](https://ourworldindata.org/internet).

## Analysis Questions

### 1. What are the top 5 countries with the highest internet use (by population share)?

In [25]:
SELECT
	entity,
	max_share
FROM
	(SELECT
		entity,
		ROUND(MAX(share), 2) AS max_share
	 FROM
	 	'internet_users.csv'
	 GROUP BY
	 	entity) AS subquery
ORDER BY
	max_share DESC
LIMIT
	5;

entity,max_share
<chr>,<dbl>
United Arab Emirates,100.00
Bahrain,99.70
Qatar,99.65
Kuwait,99.60
Liechtenstein,99.55


### 2. What is the correlation between internet usage (population share) and broadband subscriptions for 2020?

In [26]:
SELECT
	CORR(share, fixed_broadband_subs_share) AS correlation
FROM
	(SELECT
		iu.share,
	 	a.fixed_broadband_subs_share
	FROM
		'internet_users.csv' AS iu
		JOIN 
			'adoption.csv' AS a 
		ON 
			a.code=iu.code AND a.year=iu.year
	WHERE
		iu.year=2020) AS subquery;

correlation
<dbl>
0.7809321


### 3. What countries gained the most mobile subscription between 2015 and 2020?

In [27]:
WITH lag_mobile_subs AS (
	SELECT
		entity,
		year,
		mobile_cell_subs AS current_subs,
		LAG(mobile_cell_subs, 5) OVER(PARTITION BY entity ORDER BY year) AS lagged_subs
	FROM
		'adoption.csv'
	WHERE
		code IS NOT NULL AND code <> 'OWID_WRL')

SELECT
	entity,
	current_subs - lagged_subs AS mobile_subs_difference
FROM
	lag_mobile_subs
WHERE
	year = 2020
ORDER BY
	mobile_subs_difference DESC
LIMIT
	10;

entity,mobile_subs_difference
<chr>,<dbl>
China,426426700
India,152653800
Iran,53406130
Nigeria,53398570
Pakistan,49724730
Bangladesh,44903740
Myanmar,37554614
Japan,34495170
Philippines,31741340


### 4. How do the regions in the datset rank in respect to telephone landline connection, broadband internet landline connection, and mobile subscription (in terms of number of people)?

In [28]:
WITH regions AS (
	SELECT
		entity,
		fixed_telephone_subs,
		fixed_broadband_subs,
		mobile_cell_subs
	FROM
		'adoption.csv'
	WHERE
		entity LIKE '%(WB)' AND
		year=2020
)

SELECT
	entity,
	RANK() OVER(ORDER BY fixed_telephone_subs DESC) AS telephone_rank,
	RANK() OVER(ORDER BY fixed_broadband_subs DESC) AS landline_rank,
	RANK() OVER(ORDER BY mobile_cell_subs DESC) AS mobile_rank
FROM
	regions
ORDER BY
	entity;

entity,telephone_rank,landline_rank,mobile_rank
<chr>,<dbl>,<dbl>,<dbl>
East Asia and Pacific (WB),1,1,1
Europe and Central Asia (WB),2,2,3
Latin America and Caribbean (WB),4,4,5
Middle East and North Africa (WB),5,5,6
North America (WB),3,3,7
South Asia (WB),6,6,2
Sub-Saharan Africa (WB),7,7,4


### 5. Over the past ten years, what is the average population share of internet users and the average population share that has a broadband internet landline connection?

In [29]:
SELECT
	iu.year,
	AVG(iu.share) AS avg_internet_share,
	AVG(a.fixed_broadband_subs_share) AS avg_broadband_share
FROM
	'internet_users.csv' AS iu
	JOIN
		'adoption.csv' AS a
	ON
		iu.year=a.year AND iu.code=a.code
GROUP BY
	iu.year
HAVING
	iu.year BETWEEN 2011 AND 2020
ORDER BY
	iu.year

year,avg_internet_share,avg_broadband_share
<dbl>,<dbl>,<dbl>
2011,36.81949,10.82008
2012,39.49224,11.17928
2013,41.89009,11.70005
2014,44.82210,12.24641
2015,47.71957,12.86055
2016,51.13655,13.46051
2017,54.23063,14.35175
2018,58.31900,14.92934
2019,60.75806,15.28154
